Apprendre spark

In [ ]:
from pyspark.sql import SparkSession
import logging

# Reduce logging level
logging.getLogger("pyspark").setLevel(logging.ERROR)
logging.getLogger("py4j").setLevel(logging.ERROR)

# Create Spark session with quieter configuration
spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

# Set Spark's log level to ERROR (shows only errors)
spark.sparkContext.setLogLevel("ERROR")

Create DataFrame

In [ ]:
# From a list of dictionaries
data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
columns = ["name", "age"]
df_manuel = spark.createDataFrame(data, columns)

# From a CSV file
df_csv = spark.read.csv("input/file.csv", header=True, inferSchema=True)

# From JSON
df_json = spark.read.json("input/file.json")

In [ ]:
df_manuel

In [ ]:
df_csv

In [ ]:
df = df_csv

Import de base

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, max, min, year, current_date, datediff
from pyspark.sql.types import *

In [ ]:

# Initialiser Spark

print("Données originales :")
df.show()

print("Schéma :")
df.printSchema()

# 2. OPÉRATIONS DE BASE
print("\n=== OPÉRATIONS DE BASE ===")

print("Nombre total d'employés :")
print(df.count())

print("\nPremières 5 lignes :")
df.show(5)

print("\nColonnes spécifiques :")
df.select("name", "department", "salary").show()

In [ ]:
# 3. FILTRAGE
print("\n=== FILTRAGE ===")

print("Employés avec salaire > 70000 :")
df.filter(col("salary") > 70000).show()

print("Employés du département Engineering :")
df.filter(col("department") == "Engineering").show()

print("Employés de moins de 30 ans :")
df.filter(col("age") < 30).show()

In [ ]:
# 4. AGRÉGATIONS
print("\n=== AGRÉGATIONS ===")

print("Statistiques générales des salaires :")
df.agg(
    min("salary").alias("salaire_min"),
    max("salary").alias("salaire_max"),
    avg("salary").alias("salaire_moyen")
).show()

print("Nombre d'employés par département :")
df.groupBy("department").count().orderBy("count", ascending=False).show()

print("Salaire moyen par département :")
df.groupBy("department") \
  .agg(avg("salary").alias("salaire_moyen")) \
  .orderBy("salaire_moyen", ascending=False) \
  .show()

print("Age moyen par département :")
df.groupBy("department") \
  .agg(avg("age").alias("age_moyen")) \
  .show()


In [ ]:
# 5. TRANSFORMATIONS AVANCÉES
print("\n=== TRANSFORMATIONS AVANCÉES ===")

# Ajouter une colonne catégorie d'âge
from pyspark.sql.functions import when

df_with_category = df.withColumn(
    "age_category",
    when(col("age") < 30, "Jeune")
    .when(col("age") < 40, "Moyen")
    .otherwise("Senior")
)

print("Avec catégorie d'âge :")
df_with_category.select("name", "age", "age_category").show()

# Calculer l'ancienneté - VERSION SIMPLIFIÉE
df_with_tenure = df.withColumn(
    "tenure_years",
    (datediff(current_date(), col("hire_date")) / 365).cast("int")
)

print("Avec ancienneté :")
df_with_tenure.select("name", "hire_date", "tenure_years") \
               .orderBy("tenure_years", ascending=False) \
               .show()  # Pas de point ici !


In [ ]:
# 6. ANALYSES SPÉCIFIQUES
print("\n=== ANALYSES SPÉCIFIQUES ===")

print("Top 3 des salaires les plus élevés :")
df.select("name", "department", "salary") \
  .orderBy("salary", ascending=False) \
  .limit(3) \
  .show()

print("Employés embauchés en 2020 ou après :")
df.filter(year("hire_date") >= 2020) \
  .select("name", "department", "hire_date") \
  .orderBy("hire_date") \
  .show()

In [ ]:
# 7. RENOMMER LES COLONNES
print("\n=== RENOMMAGE DES COLONNES ===")

df_french = df.withColumnRenamed("name", "nom") \
              .withColumnRenamed("age", "age") \
              .withColumnRenamed("department", "departement") \
              .withColumnRenamed("salary", "salaire") \
              .withColumnRenamed("hire_date", "date_embauche")

print("DataFrame avec noms français :")
df_french.show()

In [ ]:











# 8. STATISTIQUES PAR DÉPARTEMENT
print("\n=== STATISTIQUES DÉTAILLÉES PAR DÉPARTEMENT ===")

dept_stats = df.groupBy("department").agg(
    count("*").alias("nombre_employes"),
    avg("salary").alias("salaire_moyen"),
    min("salary").alias("salaire_min"),
    max("salary").alias("salaire_max"),
    avg("age").alias("age_moyen")
)

dept_stats.show()

# Fermer Spark
# spark.stop()

In [ ]:
# Show the data
df.show()

# Display schema
df.printSchema()

# Select columns
df.select("name", "age").show()

# Filter rows
df.filter(df.age > 25).show()

